# Dependensies

In [1]:
import torch
from torch import nn

from torch.nn import functional as F

from torchsummary import summary

# Lenet-5
   - One of the pioneering convolutional neural network architectures developed in 1998 by [Yann LeCun](https://en.wikipedia.org/wiki/Yann_LeCun) and his colleagues
   - It was trained on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset (28x28 images were padded to 32x32)

<figure style="text-align: center;">
    <img src="../resources/images/SVGs/lenet5-structure.svg" alt="lenet5-structure.svg" style="width: 100%;">
    <figcaption>LeNet-5 Architecture</figcaption>
</figure>

## Custom Lenet-5
Note: `Softmax` is missing due to internal implementation of `LogSoftmax` in the `CrossEntropyLoss` function.

In [2]:
class CustomLeNet5(nn.Module):
    def __init__(self):
        super(CustomLeNet5, self).__init__()

        # input           : 1x32x32
        # output          : 6x28x28
        # trainable params: (5 * 5 + 1) * 6 = 156
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)

        # input           : 6x28x28
        # output          : 6x14x14
        # trainable params: 0
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        # input           : 6x14x14
        # output          : 16x10x10
        # trainable params: (6 * 5 * 5 + 1) * 16 = 2416
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        # input           : 16x10x10
        # output          : 16x5x5
        # trainable params: 0
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # input           : 16x5x5
        # output          : 120x1x1
        # trainable params: (16 * 5 * 5 + 1) * 120 = 48120
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)

        # input           : 120
        # output          :  84
        # trainable params: (120 + 1) * 84 = 10164
        self.fc1 = nn.Linear(in_features=120, out_features=84)

        # input           : 84
        # output          : 10
        # trainable params: (84 + 1) * 10 = 850
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # layer1:
        x = self.conv1(x)
        x = F.sigmoid(x)
        x = self.pool1(x)

        # layer2:
        x = self.conv2(x)
        x = F.sigmoid(x)
        x = self.pool2(x)

        # layer3:
        x = self.conv3(x)
        x = F.sigmoid(x)

        # flatten : 120x1x1 -> 120
        x = torch.flatten(x, 1)

        # layer4:
        x = self.fc1(x)
        x = F.sigmoid(x)

        # layer5:
        x = self.fc2(x)

        return x

In [3]:
model = CustomLeNet5()
model

CustomLeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [4]:
summary(model, (1, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         AvgPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         AvgPool2d-4             [-1, 16, 5, 5]               0
            Conv2d-5            [-1, 120, 1, 1]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
----------------------------------------------------------------
